# Self-debiasing

`Self-debiasing`과 `Human-Persona`를 적용한 코드.

`masking` 및 `few-shot`은 사용하지 않음.

실행환경: Colab T4

## 사용자 설정

In [ ]:
# 🔥하이퍼파리미터 설정
BASE_DIR = "/content/drive/MyDrive/강화학습"
CHECKPOINT_DIR = "checkpoint"
INPUT_DATA = "test.csv"
MODEL_DIR = "llama3"
DO_SAMPLE = True
TEMPERATURE = 0.6  # 커질수록 답변의 자유도가 높아집니다.
MAX_NEW_TOKENS = 512  # 너무 짧으면 답변이 끝까지 생성되지 않습니다.
TOKENIZER_MAX_LENGTH = 1024  # 너무 짧으면 프롬프트가 짤립니다.
CHECK_POINT_STEP = 100  # 몇 턴마다 체크포인트를 저장할지
NUM_WORKERS = 2  # Colab 무료 티어를 사용할 경우 2보다 키우지 마세요.
BATCH_SIZE = 32  # Cuda-Out-of-Memory가 나면 2배씩 줄이세요.
RANDOM_SEED = 33

# (int) 이전에 저장한 체크포인트
LAST_FLAG_CHECK_POINT = 0
LAST_PROMPT_CHECK_POINT = 0
LAST_SUBMISSION_CHECK_POINT = 0

In [ ]:
def generate_bias_determination(prompt) -> str:
    return (
        "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n"
        f"[프롬프트]: {prompt}\n\n"
        "먼저 프롬프트를 문장 단위로 나눈 다음, "
        "그 문장들이 판단에 영향을 줄 수 있는 인지적 편향을 포함하고 있는지 분석하세요.\n"
        "편향이 있는지 한 단어로 답하세요. (예, 아니오): <|eot_id|>"
        "<|start_header_id|>assistant<|end_header_id|>"
    )


def has_yes(response):
    for yes in ["네", "예"]:
        if yes in response:
            return True
    return False


def generate_bias_analysis(prompt) -> str:
    return (
        "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n"
        "다음은 인지적 편향을 포함하고 있을 수 있는 프롬프트입니다.\n"
        "각 문장에 어떤 인지적 편향이 포함되어 있는지 분석하고, 그 이유를 간단히 설명하세요.<|eot_id|>\n"
        f"[프롬프트]: {prompt}\n\n"
        "<|start_header_id|>assistant<|end_header_id|>"
    )


def generate_cognitive_debiasing() -> str:
    return (
        "<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n"
        "편향 판단에 따라 사람이 편향되지 않도록 프롬프트를 수정하되, "
        "원래 질문의 목적은 유지하세요.\n"
        "설명 없이 수정된 프롬프트만 출력하세요.<|eot_id|>\n"
        "<|start_header_id|>assistant<|end_header_id|>"
    )


def generate_question(prompt: str, choices: list) -> str:
    choices_with_idx = "\n".join(
        [f"{i}. {choice}" for i, choice in enumerate(choices, 1)]
    )
    return (
        "<|begin_of_text|><|start_header_id|>system<|end_header_id|>"
        "느리고 신중하게 답하는 사람처럼 행동하세요.\n"
        "그들의 답은 성의 있고 신뢰할 수 있습니다.\n"
        "이 자아를 유지하면서 아래 질문에 답하세요.<|eot_id|>"
        "<|start_header_id|>user<|end_header_id|>"
        f"[프롬프트]: {prompt.strip()}\n"
        "[선택지]:\n"
        f"{choices_with_idx}\n\n"
        "최종 답변은 설명 없이 1, 2, 3 중 하나로만 작성하세요.\n"
        "최종 답변:<|eot_id|>\n"
        "<|start_header_id|>assistant<|end_header_id|>"
    )


def trim_assistant_response(response):
    response = response.rsplit("assistant", 1)[-1]
    return response.strip("\n").strip()


def extract_last_choice(raw_answer, choices):
    # 🔥최종 답변에서 정답을 선택하세요.
    first_digit = next(
        (char for char in raw_answer if char.isdigit()), None
    )  # 처음 나오는 숫자만 추출
    if first_digit.isdigit():
        # 1 ~ 3으로 답할 경우, 정답지에서 답변 선택
        last_choice_idx = int(first_digit)
        if 1 <= last_choice_idx <= 3:
            last_choice = choices[last_choice_idx - 1]
            return last_choice

    # 이상한 답이 나올 경우, 그대로 뱉기
    raw_answer = raw_answer.strip().replace("\n", "")
    print(f"⚠️답변이 이상해요. [{raw_answer}]")
    return raw_answer

## 모델 준비

In [ ]:
import torch

assert torch.cuda.is_available(), "GPU를 사용하세요!"
device = "cuda"

In [ ]:
!pip install -qq accelerate bitsandbytes transformers

In [ ]:
import os
import ast
import gc
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from google.colab import drive


drive.mount("/content/drive", force_remount=False)


def join_path(*args):
    return os.path.join(BASE_DIR, *args)


def collect_garbage():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()


def save_csv(df, path: str, cols: list[str]):
    df[cols].to_csv(
        path,
        index=False,
        encoding="utf-8-sig",
    )

In [ ]:
# Model, Tokenizer 준비
# model_name = "meta-llama/Llama-3.1-8B-Instruct"
model_path = join_path(MODEL_DIR)

tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="left")
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

quat_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    device_map={"": 0},
    quantization_config=quat_config,
    torch_dtype=torch.float16,
)

In [ ]:
# CUDA 최적화
torch.backends.cudnn.benchmark = True
if hasattr(torch.backends.cuda, "matmul") and hasattr(
    torch.backends.cuda.matmul, "allow_tf32"
):
    torch.backends.cuda.matmul.allow_tf32 = True

# 랜덤 시드 고정
torch.manual_seed(RANDOM_SEED)
torch.cuda.manual_seed_all(RANDOM_SEED)

In [ ]:
def tokenize_batch(batch_prompts):
    return tokenizer(
        batch_prompts,
        padding=True,
        truncation=True,
        max_length=TOKENIZER_MAX_LENGTH,
        return_tensors="pt",
    ).to(device)


def generate_batch(batch_tokens, max_new_tokens):
    return model.generate(
        input_ids=batch_tokens["input_ids"],
        attention_mask=batch_tokens["attention_mask"],
        max_new_tokens=max_new_tokens,
        do_sample=DO_SAMPLE,
        temperature=TEMPERATURE,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        use_cache=True,
    )


@torch.no_grad()
def generate_response(batch_prompts, max_new_tokens):
    batch_tokens = tokenize_batch(batch_prompts)
    batch_tokens = generate_batch(batch_tokens, max_new_tokens)
    return tokenizer.batch_decode(batch_tokens, skip_special_tokens=True)

## 답변 생성

0. `context`와 `question`을 하나로 합쳐 `prompt.csv`로 저장
1. `prompt`에서 bias가 있는지 확인 후 `flag.csv`로 저장
2. bias가 있는 데이터만 뽑아 self-debiasing를 실행하고 `debiased.csv`에 저장
3. `debiased.csv`를 불러와 전체 프롬프트에 대해 답변 생성


### 0. prompt 생성

context와 question을 하나로 합칩니다.

결과: `prompt.csv`

In [ ]:
df_original = pd.read_csv(join_path(INPUT_DATA), encoding="utf-8-sig")
df_context_question = df_original[["context", "question"]]

prompts = [None] * len(df_context_question)


def concat_context_question(row):
    # context + question 합치기
    return "\n".join([row["context"], row["question"]])


with ThreadPoolExecutor(max_workers=NUM_WORKERS) as executor:
    futures = {
        executor.submit(concat_context_question, row): idx
        for idx, row in df_context_question.iterrows()
    }

    for future in as_completed(futures):
        idx = futures[future]
        prompts[idx] = future.result()

In [ ]:
df_prompt = df_original[["ID", "choices"]].copy()
df_prompt["prompt"] = prompts
save_csv(
    df_prompt,
    path=join_path("prompt.csv"),
    cols=["ID", "prompt", "choices"],
)

### 1. bias 확인

결과: `flag.csv`

In [ ]:
df_prompt = pd.read_csv(join_path("prompt.csv"), encoding="utf-8-sig")
total_data_size = len(df_prompt)

# Check point 확인
check_point_path = join_path(CHECKPOINT_DIR, f"flag_{LAST_FLAG_CHECK_POINT}.csv")
start_idx = LAST_FLAG_CHECK_POINT

if os.path.exists(check_point_path):
    df_check_point = pd.read_csv(check_point_path)
else:
    # Check point가 없을 때 초기화
    df_check_point = pd.DataFrame(
        {
            "pointer": [i for i in range(total_data_size)],
            "flag": [False for _ in range(total_data_size)],
        }
    )
    start_idx = 0

In [ ]:
@torch.no_grad()
def check_bias(batch_prompt):
    check_bias_prompt = [generate_bias_determination(prompt) for prompt in batch_prompt]
    responses = generate_response(check_bias_prompt, max_new_tokens=16)
    is_biased = [has_yes(trim_assistant_response(response)) for response in responses]
    return is_biased

In [ ]:
os.makedirs(join_path(CHECKPOINT_DIR), exist_ok=True)
collect_garbage()

# 편향 확인 시작
start_time = time.time()
while start_idx < total_data_size:
    end_idx = min(start_idx + BATCH_SIZE, total_data_size)

    batch_prompt = df_prompt.iloc[start_idx:end_idx]["prompt"].tolist()
    batch_is_biased = check_bias(batch_prompt)

    for idx, is_biased in enumerate(batch_is_biased):
        idx = idx + start_idx
        df_check_point.at[idx, "pointer"] = idx
        df_check_point.at[idx, "flag"] = is_biased

        if idx % CHECK_POINT_STEP == 0:
            # Check point에서 답변을 파일로 저장
            end_time = time.time()
            save_csv(
                df_check_point,
                path=join_path(CHECKPOINT_DIR, f"flag_{str(idx)}.csv"),
                cols=["pointer", "flag"],
            )
            print(
                f"✅{idx}/{total_data_size} 저장. ({(end_time - start_time) / 60:.1f}분)"
            )
            start_time = time.time()

    start_idx = end_idx

save_csv(
    df_check_point,
    path=join_path(CHECKPOINT_DIR, f"flag_final.csv"),
    cols=["pointer", "flag"],
)

In [ ]:
# 편향이 있다고 판단한 prompt만 모아서 저장
df_flag = pd.concat([df_prompt, df_check_point], axis=1)
df_flag = df_flag[df_flag["flag"].astype(bool) == True]
save_csv(df_flag, path=join_path("flag.csv"), cols=["pointer", "prompt"])

print(f"편향이 있다고 판단한 프롬프트: {len(df_flag)}개")

### 2. prompt 재생성

결과: `debiased.csv`

In [ ]:
df_flag = pd.read_csv(join_path("flag.csv"))

# Check point 확인
check_point_path = join_path(CHECKPOINT_DIR, f"debiased_{LAST_PROMPT_CHECK_POINT}.csv")
start_idx = LAST_PROMPT_CHECK_POINT

if os.path.exists(check_point_path):
    df_check_point = pd.read_csv(check_point_path)
else:
    # Check point가 없을 때 초기화
    df_check_point = df_flag
    df_check_point[["debiased", "reason"]] = ""
    start_idx = 0

del df_flag
total_data_size = len(df_check_point)

In [ ]:
@torch.no_grad()
def generate_debiased_prompt(batch_prompt):
    check_bias_prompt = [generate_bias_analysis(prompt) for prompt in batch_prompt]
    check_bias_responses = generate_response(
        check_bias_prompt, max_new_tokens=MAX_NEW_TOKENS
    )
    remove_bias_prompt = generate_cognitive_debiasing()
    remove_bias_prompt = [
        f"{response}\n{remove_bias_prompt}" for response in check_bias_responses
    ]
    remove_bias_responses = generate_response(
        remove_bias_prompt, max_new_tokens=MAX_NEW_TOKENS
    )

    return remove_bias_responses

In [ ]:
os.makedirs(join_path(CHECKPOINT_DIR), exist_ok=True)
collect_garbage()

# 편향 없는 프롬프트로 재생성
start_time = time.time()
while start_idx < total_data_size:
    end_idx = min(start_idx + BATCH_SIZE, total_data_size)

    batch_context = df_check_point.iloc[start_idx:end_idx]["prompt"].tolist()
    debiased_prompts = generate_debiased_prompt(batch_context)

    for idx, debiased_prompt in enumerate(debiased_prompts):
        idx = idx + start_idx
        reason, prompt = debiased_prompt.rsplit("assistant", 1)
        df_check_point.at[idx, "debiased"] = prompt
        df_check_point.at[idx, "reason"] = reason

        if idx % CHECK_POINT_STEP == 0:
            # Check point에서 답변을 파일로 저장
            end_time = time.time()
            save_csv(
                df_check_point,
                path=join_path(CHECKPOINT_DIR, f"debiased_{str(idx)}.csv"),
                cols=["pointer", "prompt", "debiased", "reason"],
            )
            print(
                f"✅{idx}/{total_data_size} 저장. ({(end_time - start_time) / 60:.1f}분)"
            )
            start_time = time.time()

    start_idx = end_idx

save_csv(
    df_check_point,
    path=join_path(CHECKPOINT_DIR, "debiased_final.csv"),
    cols=["pointer", "prompt", "debiased", "reason"],
)

In [ ]:
df_prompt = pd.read_csv(join_path("prompt.csv"), encoding="utf-8-sig")

for _, row in df_check_point.iterrows():
    idx = row["pointer"].astype(int)
    debiased = row["debiased"]
    df_prompt.at[idx, "prompt"] = debiased

save_csv(
    df_prompt,
    path=join_path("debiased.csv"),
    col=["ID", "prompt", "choices"],
)

### 3. 최종 답변 추론

In [ ]:
# 질문 데이터 준비
df_prompt = pd.read_csv(join_path("debiased.csv"))

# Check point 확인
check_point_path = join_path(
    CHECKPOINT_DIR, f"submission_{LAST_SUBMISSION_CHECK_POINT}.csv"
)
start_idx = LAST_PROMPT_CHECK_POINT

if os.path.exists(check_point_path):
    df_check_point = pd.read_csv(check_point_path)
else:
    # Check point가 없을 때 초기화
    df_check_point = df_prompt
    start_idx = 0
    for col in ["raw_input", "raw_output", "answer"]:
        df_check_point[col] = ""

total_data_size = len(df_check_point)

In [ ]:
@torch.no_grad()
def inference(batch_data):
    question_prompt = [
        generate_question(data["prompt"], ast.literal_eval(data["choices"]))
        for data in batch_data
    ]
    responses = generate_response(question_prompt, max_new_tokens=16)
    return responses

In [ ]:
os.makedirs(join_path(CHECKPOINT_DIR), exist_ok=True)
collect_garbage()

# 모델 추론 시작
start_time = time.time()
while start_idx < total_data_size:
    end_idx = min(start_idx + BATCH_SIZE, total_data_size)

    batch_df = df_check_point.iloc[start_idx:end_idx][["prompt", "choices"]]
    responses = inference(batch_df)

    for idx, response in enumerate(responses):
        idx = idx + start_idx
        raw_input, raw_answer = response.rsplit("assistant", 1)
        df_check_point.at[idx, "raw_input"] = raw_input
        df_check_point.at[idx, "raw_output"] = raw_answer
        choices = ast.literal_eval(df_original.at[idx, "choices"])
        df_check_point.at[idx, "answer"] = extract_last_choice(raw_answer, choices)

        if idx % CHECK_POINT_STEP == 0:
            # Check point에서 답변을 파일로 저장
            end_time = time.time()
            save_csv(
                df_check_point,
                path=join_path(CHECKPOINT_DIR, f"submission_{str(idx)}.csv"),
                cols=["ID", "raw_input", "raw_output", "answer"],
            )
            print(
                f"✅{idx}/{total_data_size} 저장. ({(end_time - start_time) / 60:.1f}분)"
            )
            start_time = time.time()

    start_idx = end_idx

## 제출 파일 저장

In [ ]:
# 최종 파일 저장
save_csv(
    df_check_point,
    join_path("submission.csv"),
    cols=["ID", "raw_input", "raw_output", "answer"],
)
print("🫠기록이 완료되었습니다.")